In [ ]:
# The modules we're going to use
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation, pooling, Flatten, Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import Adam
from keras import callbacks
from sklearn import preprocessing

import numpy as np
np.random.seed(1337)
import matplotlib.pyplot as plt

import csv

# When you execute a code to plot with a simple SHIFT-ENTER, the plot will be shown directly under the code cell
%matplotlib inline

# Set backend data format
from keras import backend as K
print('Default format:', K.image_data_format())
K.set_image_data_format('channels_first')
print('Current format:', K.image_data_format())


In [ ]:
# functions to load/save data
def load_data(file_name):
    data = []
    with open(file_name) as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        for row in reader:
            data.append([int(x) for x in row])
    return data
    
def load_feature(file_name):
    return np.array(load_data(file_name), 'float32')

def load_target(file_name):
    targets = load_data(file_name)
    # class should start from 0
    num_classes = np.max(targets) + 1
    return np_utils.to_categorical(targets, num_classes)

def save_result(result, file_name='test_target.csv'):
    with open(file_name, 'w') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        writer.writerow(['Id', 'Category']) # Header
        for i in range(len(result)): # Targets
            writer.writerow([i, result[i]])

    

In [ ]:
# Load data
file_X = 'data/train_data.csv'
file_y = 'data/train_target.csv'
X = load_feature(file_X)
y = load_target(file_y)

# Data scaling
scaler = preprocessing.StandardScaler().fit(X)
X = scaler.transform(X)

# Pre-processing
print('Before pre-processing, X_train size: ', X.shape)
X = X.reshape(-1, 1, 48, 48)
print('After pre-processing, X_train size: ', X.shape)



In [ ]:
# Model design
def createModel():
    # Define dropout rate
    dprate = 0.5;

    # Create a neural net
    model = Sequential()
    
    # Convolutional layers
    model.add(Conv2D(64, (3,3), strides=1, padding='same', activation='relu', input_shape=(1,48,48), name='c64_1'))
    model.add(Conv2D(64, (3,3), strides=1, padding='same', activation='relu', name='c64_2'))
    model.add(MaxPooling2D(pool_size=2, strides=2, padding='valid', name='mp64'))
    
    model.add(Conv2D(128, (3,3), strides=1, padding='same', activation='relu', name='c128_1'))
    model.add(Conv2D(128, (3,3), strides=1, padding='same', activation='relu', name='c128_2'))
    model.add(MaxPooling2D(pool_size=2, strides=2, padding='valid', name='mp128'))

    model.add(Conv2D(256, (3,3), strides=1, padding='same', activation='relu', name='c256_1'))
    model.add(Conv2D(256, (3,3), strides=1, padding='same', activation='relu', name='c256_2'))
    model.add(MaxPooling2D(pool_size=2, strides=2, padding='valid', name='mp256'))

    # Fully connected layer
    model.add(Flatten(name='flatten'))
    model.add(Dense(1024, activation='relu', name='d1024_1'))
    model.add(Dropout(dprate, name='dp_1'))
    model.add(Dense(1024, activation='relu', name='d1024_2'))
    model.add(Dropout(dprate, name='dp_2'))
    model.add(Dense(3, activation='softmax', name='d3'))
    
    # Specify an optimizer to use
    adam = Adam(lr=0.0002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

    # Choose loss function, optimization method, and metrics (which results to display)
    model.compile(
        optimizer = adam,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model
    

In [ ]:
# Training
model = createModel()
# early_stop = callbacks.EarlyStopping(monitor='val_acc', patience=2, verbose=2)
# history = model.fit(X,y,epochs=100,batch_size=32,validation_split=0.1, callbacks=[early_stop], verbose=2)
history = model.fit(X,y,epochs=8,batch_size=32,verbose=2)


In [ ]:
# load predicting data
file_X_predict = 'data/test_data.csv'
X_predict = load_feature(file_X_predict)

# Data scaling
X_predict = scaler.transform(X_predict)

# Pre-processing
X_predict = X_predict.reshape(-1,1, 48,48)
print('X predict size: ', X_predict.shape)

In [ ]:
# Predicting
y_predict = model.predict(X_predict)

# convert probs to class
y_result = [np.argmax(probs) for probs in y_predict]

# Save predicting result to file
save_result(y_result, 'data/test_target.csv')


In [ ]:
# Show the image of one predicting example
index = np.random.randint(X_predict.shape[0],size=1)
plt.imshow(X_predict[index[0],0,:,:],cmap='gray')

# Get its prediction
output = y_predict[index[0]]
print("Probs: ", output)
print("Class: ", y_result[index[0]])
plt.figure()
plt.xticks(np.arange(output.shape[0]))
plt.plot(np.arange(output.shape[0]),output)


In [ ]:
# Persist model
model.save('data/model_642_1282_2562-10242.h5')
model.save_weight2('data/weights_642_1282_2562-10242.h5')